In [ ]:
import base64
import io
import requests
from urllib.parse import urljoin
from tqdm import notebook
import time
import yaml
from PIL import Image
import json
import re

from abeja.datalake import APIClient
from abeja.datalake import Client as DatalakeClient

In [ ]:
ANNOTATION_JSON_NAME = 'XXXXXXXXXXXX.json'
api_client = APIClient()

page = 1
delay = 3

# get credential information
yaml_dict = yaml.load(open('secret.yaml').read(), Loader=yaml.SafeLoader)

# set credential
organization_id = yaml_dict['organization_id']
user_id = yaml_dict['user_id']
personal_access_token = yaml_dict['personal_access_token']

credential = {
    'user_id': user_id,
    'personal_access_token': personal_access_token
}

# set credential for preinference
annotation_user_id = yaml_dict['annotation_user_id']
annotation_access_token = yaml_dict['annotation_access_token']
annotation_organization_id = yaml_dict['annotation_organization_id']
annotation_project_id = yaml_dict['annotation_project_id']

headers = {
    'api-access-user-id': annotation_user_id,
    'api-access-token': annotation_access_token
}

# get DataLake channel id in and out
annotation_api = 'https://annotation-tool.abeja.io'
project_url = urljoin(annotation_api, '/api/v1/organizations/{}/projects/{}'.format(annotation_organization_id,
                                                                                    annotation_project_id))
res_project_url = requests.get(project_url, headers=headers)
res_project_url.raise_for_status()
channel_id_in = res_project_url.json()['data_lake_channels'][0]['channel_id']
channel_id_out = res_project_url.json()['result_data_lake_channel']['channel_id']

In [ ]:
while True:
        # get annotation tasks
        task_url = urljoin(annotation_api, '/api/v1/organizations/{}/projects/{}/tasks/'.format(
            annotation_organization_id, annotation_project_id))
        res_task_url = requests.get(task_url,
                                    headers=headers,
                                    params={'page': page})
        res_task_url.raise_for_status()

        # check if project has data
        if len(res_task_url.json()) == 0:
            break

        f = open(ANNOTATION_JSON_NAME,'r')
        json_dict = json.load(f)
            
        for task in tqdm(res_task_url.json()):
            # load image from DataLake channel-in
            client = DatalakeClient(organization_id=organization_id, credential=credential)
            channel = client.get_channel(channel_id_in)
            metadata = task['metadata'][0]
            input_img = channel.get_file(metadata['file_id'])
            content_type = input_img.get_file_info()['content_type']
            input_img_io = io.BytesIO(input_img.get_content())
            input_img_fike_id = metadata['file_id']

            information = []
            
            for x in json_dict:
                search_data = x['task']['metadata']
                search_in = [d for d in search_data if 'source' in d]
                search_str = ','.join(map(str,search_in))
                search_name_pick = search_str.split(',', 15)[3]
                search_name = search_name_pick.split(':', 2)[1]
                search = search_name.strip("'"" '")
                
                if input_img_fike_id == search:
                        
                    annotation_data = x['information']

            
                    origin_in = [d for d in annotation_data ]
                    origin_str = ','.join(map(str,origin_in))
                
                    annotation_data.pop()
                
                    for s in annotation_data:
                        i = 0    
                        if 'id' in s:
                                       
                            file_id_pick_str = s['file_id']
                
                            # get file url
                            file_info_url = 'https://api.abeja.io//channels/{}/{}'.format(channel_id_out, file_id_pick_str)
                            res_file_info_url = requests.get(file_info_url,
                                                     auth=(user_id, personal_access_token))
                            res_file_info_url.raise_for_status()
                            file_url = res_file_info_url.json()['download_url']

                            # information_list
                            information.append({
                                'class': s['class'],
                                'color': s['color'],
                                'file_id': s['file_id'],
                                'file_url': file_url,
                                'id': s['id']}
                            )
                            
                    # register predicted result to annotation tool
                    preinference_url = urljoin(task_url, "{}/preinferences".format(str(task['id'])))
                    res_preinference_url = requests.post(preinference_url,
                                                            json={'information': information},
                                                            headers=headers)
                    res_preinference_url.raise_for_status()
                    
                    #print(res_preinference_url.json())
                    time.sleep(delay)
                                    
                else:
                    pass

        page = page + 1
